# Elliptic Curve

equation:
$$y^2 = x^3 + ax + b  \thinspace mod \thinspace    P $$
$$ y,x,a,b   \thinspace \epsilon   \thinspace F_P $$

$y, x, a, b$ integers modulo $P$


<img src="image/curve.png"/>

## Point Operation
two basic operations are:
 * Point addition: $R = P + Q$
 * Point doubling: $R = P + P$
 
### Point Addition

$$ R = P + Q = \binom{P_x}{P_y} + \binom{Q_x}{Q_y}, P \neq Q $$

__for this we need to calculate Algebraic:__

If P and Q are distinct $(P_x \neq Q_x)$, the line through them has slope:
$$ m=P_y−Q_y P_x−Q_x $$

The intersection of this line with the elliptic curve is a third point $R = \binom{R_x}{R_y}$:
$$R_x =m^2−P_x−Q_x \\ R_y = P_y +m(R_x−P_x)$$
or, equivalently:
$$R_y=Q_y +m(R_x−Q_x)$$

Hence $\binom{P_x}{P_y}+\binom{Q_x}{Q_y}=\binom{R_x}{−R_y}$ (note that: $P+Q=−R$).




<img src="image/addition.jpeg" />

### Point Doubling


$$  R = P + P  $$ 

__for this we need to calculate:__

$$  m = \frac{3P_x^2+a}{2 P_y}  $$

<img  src="image/double.png" />

## Python Code

first of all we intialize variables.

Here, Initialized keys are based on 
<a href="https://en.bitcoin.it/wiki/Secp256k1"> secp256k1</a>, parameters of the elliptic curve used in Bitcoin's public-key cryptography 

In [1]:
import collections
import random
import hashlib
import random

# namedtuple: The standard tuple uses numerical indexes to access its members
# for example: when create new ECC, it's inputs set to variables in order of definition
ECC = collections.namedtuple('ECC', 'p a b g n')

curve = ECC(
    # Field characteristic.
    p=0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f,
    # Curve coefficients.
    a=0,
    b=7,
    # Base point.
    g=(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
       0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8),
    # Subgroup order.
    n=0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
)

inv_mod uses <a href="https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm">Extended Euclidean algorithm</a>
it returns inverse of k modulo p:
$$ (x*k) \thinspace mod \thinspace p \thinspace = \thinspace 1 $$

In [2]:
def inv_mod(k, p):
    """Returns the inverse of k modulo p.
    k must be non-zero and p must be a prime.
    """
    if k == 0:
        raise ZeroDivisionError('division by zero')

    if k < 0:
        # k ** -1 = p - (-k) ** -1  (mod p)
        return p - inv_mod(-k, p)

    # Extended Euclidean algorithm.
    s, old_s = 0, 1
    t, old_t = 1, 0
    r, old_r = p, k

    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t

    gcd, x, y = old_r, old_s, old_t

    assert gcd == 1
    assert (k * x) % p == 1

    return x % p

In [3]:
def is_on_curve(point):
    """Returns True if the given point lies on the elliptic curve."""
    if point is None:
        # None represents the point at infinity.
        return True

    x, y = point

    return (y * y - x * x * x - curve.a * x - curve.b) % curve.p == 0


def ecc_neg(point):
    """Returns -point."""
    assert is_on_curve(point)

    if point is None:
        # -0 = 0
        return None

    x, y = point
    result = (x, -y % curve.p)

    assert is_on_curve(result)

    return result



In [4]:
def ecc_add(point1, point2):
    """
        Returns the result of point1 + point2 according to the group law.
    """
    assert is_on_curve(point1)
    assert is_on_curve(point2)

    if point1 is None:
        return point2
    if point2 is None:
        return point1

    x1, y1 = point1
    x2, y2 = point2

    if x1 == x2 and y1 != y2:
        return None

    m = (y1 - y2) * inv_mod(x1 - x2, curve.p)

    x3 = m * m - x1 - x2
    y3 = y1 + m * (x3 - x1)
    result = (x3 % curve.p,-y3 % curve.p)

    assert is_on_curve(result)

    return result

def ecc_double(point1):
    """
        Returns the result of doubling point1 according to the group law.
    """
    assert is_on_curve(point1)

    x1, y1 = point1

    m = (3 * x1 * x1 + curve.a) * inv_mod(2 * y1, curve.p)

    x3 = m * m - 2 * x1
    y3 = y1 + m * (x3 - x1)
    result = (x3 % curve.p,
              -y3 % curve.p)

    assert is_on_curve(result)

    return result



def ecc_mul(k, point):
    """Returns k * point computed using the ecc_double and ecc_add algorithm."""
    assert is_on_curve(point)

    if k % curve.n == 0 or point is None:
        return None

    if k < 0:
        return ecc_mul(-k, ecc_neg(point))

    result = None
    preres = point

    while k:
        if k & 1: # if first bit is 1
            # Add.
            result = ecc_add(result, preres)

        # Double.
        preres = ecc_double(preres)

        k >>= 1

    assert is_on_curve(result)

    return result

### Discrete Logarithm Problem
if we know points $P$ and $Q$, what is scalar $k$ such that $Q=kP$?

### ECDH
Here's how it works:

__First__, Alice and Bob generate their own private and public keys. We have the private key $d_A$ and the public key $H_A=d_AG$ for Alice, and the keys d_B and H_B=d_BG for Bob. Note that both Alice and Bob are using the same domain parameters: the same base point $G$ on the same elliptic curve on the same finite field.

__Second__, Alice and Bob exchange their public keys $H_A$ and $H_B$ over an insecure channel. Eve would intercept $H_A$ and $H_B$, but won't be able to find out neither $d_A$ nor $d_B$ without solving the discrete logarithm problem.

__Third__, Alice calculates $S=d_AH_B$ , and Bob calculates $S=d_BH_A$.  $S$ is the same for both Alice and Bob, because:
$$S=d_AH_B=d_A(d_BG)=d_B(d_AG)=d_BH_A$$
Eve only knows $H_A$ and $H_B$  and would not be able to find out the shared secret $S$. This is known as the `Diffie-Hellman` problem.

In [5]:
def make_keypair():
    """Generates a random private-public key pair."""
    private_key = random.randrange(1, curve.n)
    public_key = ecc_mul(private_key, curve.g)

    return private_key, public_key



# Alice generates her own keypair.
a_prv_key, a_pub_key = make_keypair()
print("Alice's private key:", hex(a_prv_key))
print("Alice's public key: (0x{:x}, 0x{:x})".format(*a_pub_key))
print()

# Bob generates his own key pair.
b_prv_key, b_pub_key = make_keypair()
print("Bob's private key:", hex(b_prv_key))
print("Bob's public key: (0x{:x}, 0x{:x})".format(*b_pub_key))
print()

# Alice and Bob exchange their public keys and calculate the shared secret.
s1 = ecc_mul(a_prv_key, b_pub_key)
s2 = ecc_mul(b_prv_key, a_pub_key)
assert s1 == s2

print('Shared secret: (0x{:x}, 0x{:x})'.format(*s1))

Alice's private key: 0x583d394a4a6c7dede8206c72f38628ad47cdf69516292260a0c6c44bd127c881
Alice's public key: (0x21f87ff2fdba3c58540913c013214711c504642891823799533aa9dd5309d84e, 0xcc20aa63a611cd42e98d44d214dcea62d87179bd20a66e58e5c24106765e24f5)

Bob's private key: 0x730b560368048e1379dbd1937feb551d77393f3d7c1dfba953e50398cfda292a
Bob's public key: (0x41265f7465d564f6d7a2f3281a2c23e1b14901aa3bad214b84308b1a19e6f678, 0x74551679f5280c8089f9438d1367850b3534940dc1514031826d16a4f7df6b13)

Shared secret: (0x611140b0720fa5a7a3cd31614ad7036aea628fa4ae5186e712a0e2188c7a6b9, 0x4916b86e56c7f2f4a2975c85dda3682c3637322b4eb5fc5d19d7457952106669)


### ECDSA
 Alice wants to sign a message with her private key $(d_A)$, and Bob wants to validate the signature using Alice's public key $(H_A)$. Nobody but Alice should be able to produce valid signatures. Everyone should be able to check signatures.

The algorithm performed by Alice to sign the message works as follows:

__1.__ Take a random integer k chosen from ${1,…,n−1}$ (where $n$ is still the subgroup order).<br/>
__2.__ Calculate the point $P=kG$ (where $G$ is the base point of the subgroup).<br/>
__3.__ Calculate the number $r=x_P \thinspace mod \thinspace n$ (where $x_P$ is the $x$ coordinate of $P$).<br/>
__4.__ If $r=0$, then choose another $k$ and try again.<br/>
__5.__ Calculate $s=k−1(z+rd_A) \thinspace mod \thinspace n $(where $d_A$ is Alice's private key and $k−1$ is the multiplicative inverse of $k$ modulo $n$).<br/>
__6.__ If $s=0$, then choose another $k$ and try again.<br/>

In [6]:
def hash_message(message):
    """Returns the truncated SHA512 hash of the message."""
    message_hash = hashlib.sha512(message).digest()
    e = int.from_bytes(message_hash, 'big')

    # truncate hash
    z = e >> (e.bit_length() - curve.n.bit_length())

    assert z.bit_length() <= curve.n.bit_length()

    return z



def sign_message(private_key, message):
    z = hash_message(message)

    r = 0
    s = 0

    while not r or not s:
        k = random.randrange(1, curve.n)
        x, y = ecc_mul(k, curve.g)

        r = x % curve.n
        s = ((z + r * private_key) * inv_mod(k, curve.n)) % curve.n

    return (r, s)


def verify_signature(public_key, message, signature):
    z = hash_message(message)

    r, s = signature

    w = inv_mod(s, curve.n)
    u1 = (z * w) % curve.n
    u2 = (r * w) % curve.n

    x, y = ecc_add(ecc_mul(u1, curve.g),
                     ecc_mul(u2, public_key))

    if (r % curve.n) == (x % curve.n):
        return 'signature matches'
    else:
        return 'invalid signature'



private, public = make_keypair()
print("Private key:", hex(private))
print("Public key: (0x{:x}, 0x{:x})".format(*public))

msg = b'Hello!'
signature = sign_message(private, msg)

print()
print('Message:', msg)
print('Signature: (0x{:x}, 0x{:x})'.format(*signature))
print('Verification:', verify_signature(public, msg, signature))

msg = b'Hi there!'
print()
print('Message:', msg)
print('Verification:', verify_signature(public, msg, signature))

private, public = make_keypair()

msg = b'Hello!'
print()
print('Message:', msg)
print("Public key: (0x{:x}, 0x{:x})".format(*public))
print('Verification:', verify_signature(public, msg, signature))

Private key: 0xd5bae44b3de577569fafe41587f2d69de7fe333876ca772fba56039cf8d595d6
Public key: (0x519fd4e150ec84315090d11334669208b7618f29ed61c3306cb724e346f689a4, 0x58c385b1cf3669fc43d324be12a35910c8224fda619b1b47c7d68022dce756aa)

Message: b'Hello!'
Signature: (0xbb6074a35f6f5f9bab0b8df00af4e60202b08a4bcce1d6943cdb69e1b22528c3, 0x64cb46ce425bd395f21d69f1f9fb20747df682ff7e6d733c7a5d1a20c3d2ccb8)
Verification: signature matches

Message: b'Hi there!'
Verification: invalid signature

Message: b'Hello!'
Public key: (0xff19cd343d26d34d64cb6a393a2151f786accb55e0f309e89c7bc4d8436d11e6, 0xb947f742c52f6bbb93339785012fa16cf9bd100c13ace2ef5dbc17b1839003d6)
Verification: invalid signature
